In [1]:
import json
import os
import pandas as pd
from difflib import SequenceMatcher
import jellyfish
from geopy.geocoders import Nominatim
geolocator = Nominatim()
from geopy.distance import great_circle

In [2]:
def tryGeoCode(value):
    try:
        print(value)
        return geolocator.geocode(value)
    except:
        return None

In [3]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def similar_jelly(a,b):
    a = a.upper()
    b = b.upper()
    return jellyfish.jaro_distance(a,b)

In [4]:
def multipleReplace(text, wordDict):
    for key in wordDict:
        text = text.replace(key, wordDict[key])
    return text

In [5]:
wordDict = {' los ':' ',' al ':' ',' de ':' ','Estatua de ':'','Monumento a ':'','Restos del ':'','Estatua a ':'','Silo de ':'', 'Puerta de ':'','Estatua':'','Monumento':'', ' a ':' '}

In [6]:
def inside(coor, bbox):
    try:
        lat = float(coor[0])
        lon =  float(coor[1])
        bbox_lat0 =  float(bbox[0])
        bbox_lat1 =  float(bbox[1])
        bbox_lon0 =  float(bbox[2])
        bbox_lon1 =  float(bbox[3])
        if lat > bbox_lat0 and lat < bbox_lat1 and lon > bbox_lon0 and lon < bbox_lon1:
            return True
        else:
            return False
    except:
        return "not valid"

In [7]:
def getDistance_score(x1, x2):
    try:
        return 100/(great_circle(x1, x2).kilometers*1000.0)
    except:
        pass

def getDistance_meters(x):
    print(x)
    try:
        return great_circle(x[0], x[1]).kilometers*1000.0
    except:
        return 0

In [8]:
def tryAssign_point(x):
    try:
        lat = x.latitude
        lon = x.longitude
        return lat,lon
    except:
        print(x,' Not valid')

In [9]:
def getName(titulo):
    for item in range(len(data['features'])):
        try:
            if similar_jelly(titulo.upper(),multipleReplace(data['features'][item]['properties']['name'], wordDict) )>0.75:
                return int(item)
        except:
            pass
        
def getName_score(titulo):
    for item in range(len(data['features'])):
        try:
            if similar_jelly(titulo.upper(),multipleReplace(data['features'][item]['properties']['name'], wordDict) )>0.75:
                return similar_jelly(titulo,multipleReplace(data['features'][item]['properties']['name'], wordDict))
        except:
            pass      


In [10]:
files = os.listdir()

In [11]:
ayuntamiento = pd.read_excel('anexo i monumentos (1).xls', skiprows=1, header=None)
ayuntamiento.columns=['COD','TIPO','CALIF','TITULO','LOCALIZACION']
ayuntamiento = ayuntamiento.set_index('COD')
ayuntamiento['OSM'] = 0
ayuntamiento['OSM_score'] = 0
ayuntamiento['OSMLocation'] = None
ayuntamiento['OSMLocation_score'] = None
ayuntamiento['COORD'] = 0
ayuntamiento['COORD_OSM'] = 0

In [12]:
with open('export (2).geojson') as f:
    data = json.load(f)

In [13]:
ayuntamiento.OSM = ayuntamiento.TITULO.apply(lambda x: getName(x))
ayuntamiento.OSM_score = ayuntamiento.TITULO.apply(lambda x: getName_score(x))

In [14]:
test = ayuntamiento.sort_values('OSM_score', ascending =False)[:50]

In [20]:
test.head()

,TIPO,CALIF,TITULO,LOCALIZACION,OSM,OSM_score,OSMLocation,OSMLocation_score,COORD,COORD_OSM
COD,,,,,,,,,,
8799,5,NOT,MIGUEL MOYA,"SERRANO, 4",86.0,1.000000,"(El Serrano, Almería, Andalucía, 04001 - 04009...",None,"(36.9613047, -2.2817453)","(40.4111859, -3.6793879)"
8552,1,NaN,JAIME I,FRANCISCO Y JACINTO ALCANTARA,56.0,1.000000,"(Calle de Francisco y Jacinto Alcántara, Parqu...",None,"(40.4295405, -3.7238498)","(40.429222, -3.7239872)"
9574,3,NaN,CABEZA OLMECA,C/VALDERREBOLLO/C/ GRANJA DE SAN ILDEFONSO/,59.0,1.000000,None,None,None,"(40.3708906, -3.627407)"
9096,8,NOT,GRUTA DEL LABERINTO,PARQUE EL CAPRICHO,96.0,1.000000,"(Parque de El Capricho, Barrio del Aeropuerto,...",None,"(40.4563176, -3.59797462561085)","(40.4561933, -3.5959202)"
8016,1,NaN,CARLOS III,JARDINES SABATINI,10.0,0.969697,"(Jardines de Sabatini, Cuesta de San Vicente, ...",None,"(40.4204954, -3.71393477143546)","(40.4111512, -3.6907121)"


In [15]:
data['features'][20]['geometry']['coordinates']

[-3.6753031, 40.4248513]

In [16]:
test.COORD_OSM = test.OSM.apply(lambda x: (data['features'][int(x)]['geometry']['coordinates'][1],data['features'][int(x)]['geometry']['coordinates'][0]))

In [17]:
test.OSMLocation = test.LOCALIZACION.apply(lambda x: tryGeoCode(x+', MADRID'))

SERRANO, 4, MADRID
FRANCISCO Y JACINTO ALCANTARA, MADRID
C/VALDERREBOLLO/C/ GRANJA DE SAN ILDEFONSO/, MADRID
PARQUE EL CAPRICHO, MADRID
JARDINES SABATINI, MADRID
PUERTA DEL SOL, MADRID
BULEVAR DE LA NATURALEZA Nº40 (SUR), MADRID
PARQUE DEL RETIRO, (Pº. ARGENTINA), MADRID
ALCALA C/V GOYA, MADRID
FRANCISCO Y JACINTO ALCANTARA (ANT. PASO A NIVEL), MADRID
MAR DE LAS ANTILLAS, 2 (PARQUE HUERTA DE LA SALUD), MADRID
C/ BAILEN (JARDINES DE SABATINI), MADRID
Pº. CAMOENS, (PARQUE DEL OESTE), MADRID
AV. ALBUFERA (METRO MIGUEL HERNANDEZ), MADRID
Pº. RUPERTO CHAPI, (PARQUE DEL OESTE), MADRID
SERRADILLA, 13 (C.C.MUNIC. "MIGUEL HERNANDEZ"), MADRID
Pº. CASTELLANA, (JARD. MUSEO CIENCIAS NATURALES), MADRID
Pº. CAMOENS, (PARQUE DEL OESTE), MADRID
CASA DE CAMPO/CRA. DE BOADILLA, MADRID
PZA. SANTA ANA, MADRID
PARQUE EL CAPRICHO, MADRID
Pº. CASTELLANA, (JARD. MUSEO CIENCIAS NATURALES, MADRID
PZA. DE COLON - GENOVA, MADRID
PZA. DE ORIENTE, MADRID
GLORIETA PUERTA DE TOLEDO, MADRID
PARQUE DE BERLIN, (ESTANQUE)

In [18]:
test.COORD = test.OSMLocation.apply(lambda x: tryAssign_point(x))

None  Not valid
None  Not valid
None  Not valid
None  Not valid
None  Not valid
None  Not valid
None  Not valid
None  Not valid
None  Not valid
None  Not valid
None  Not valid
None  Not valid
None  Not valid
None  Not valid
None  Not valid
None  Not valid
None  Not valid
None  Not valid
None  Not valid
None  Not valid
None  Not valid
None  Not valid
None  Not valid
None  Not valid
None  Not valid
None  Not valid
None  Not valid


In [19]:
test

,TIPO,CALIF,TITULO,LOCALIZACION,OSM,OSM_score,OSMLocation,OSMLocation_score,COORD,COORD_OSM
COD,,,,,,,,,,
8799,5,NOT,MIGUEL MOYA,"SERRANO, 4",86.0,1.000000,"(El Serrano, Almería, Andalucía, 04001 - 04009...",None,"(36.9613047, -2.2817453)","(40.4111859, -3.6793879)"
8552,1,NaN,JAIME I,FRANCISCO Y JACINTO ALCANTARA,56.0,1.000000,"(Calle de Francisco y Jacinto Alcántara, Parqu...",None,"(40.4295405, -3.7238498)","(40.429222, -3.7239872)"
9574,3,NaN,CABEZA OLMECA,C/VALDERREBOLLO/C/ GRANJA DE SAN ILDEFONSO/,59.0,1.000000,None,None,None,"(40.3708906, -3.627407)"
9096,8,NOT,GRUTA DEL LABERINTO,PARQUE EL CAPRICHO,96.0,1.000000,"(Parque de El Capricho, Barrio del Aeropuerto,...",None,"(40.4563176, -3.59797462561085)","(40.4561933, -3.5959202)"
8016,1,NaN,CARLOS III,JARDINES SABATINI,10.0,0.969697,"(Jardines de Sabatini, Cuesta de San Vicente, ...",None,"(40.4204954, -3.71393477143546)","(40.4111512, -3.6907121)"
8974,1,NOT,CARLOS III,PUERTA DEL SOL,10.0,0.969697,"(Puerta del Sol, Calle del Carmen, Barrio Sol,...",None,"(40.4170316, -3.7034288)","(40.4111512, -3.6907121)"
9649,6,NaN,ARBOL MEDIÁTICO,BULEVAR DE LA NATURALEZA Nº40 (SUR),60.0,0.955556,None,None,None,"(40.3700085, -3.6301475)"
8261,1,NOT,CARLOS II,"PARQUE DEL RETIRO, (Pº. ARGENTINA)",10.0,0.939394,None,None,None,"(40.4111512, -3.6907121)"
9189,1,NaN,GOYA,ALCALA C/V GOYA,20.0,0.933333,None,None,None,"(40.4248513, -3.6753031)"


In [207]:
lista = []
for x,y in zip(test.COORD.values, test.COORD_OSM.values):
    lista.append(getDistance_score(x,y))
test.OSMLocation_score = lista

In [208]:
test

,TIPO,CALIF,TITULO,LOCALIZACION,OSM,OSM_score,OSMLocation,OSMLocation_score,COORD,COORD_OSM
COD,,,,,,,,,,
8799,5,NOT,MIGUEL MOYA,"SERRANO, 4",86.0,1.000000,"(El Serrano, Almería, Andalucía, 04001 - 04009...",0.000248,"(36.9613047, -2.2817453)","(40.4111859, -3.6793879)"
8552,1,NaN,JAIME I,FRANCISCO Y JACINTO ALCANTARA,56.0,1.000000,"(Calle de Francisco y Jacinto Alcántara, Parqu...",2.681920,"(40.4295405, -3.7238498)","(40.429222, -3.7239872)"
9574,3,NaN,CABEZA OLMECA,C/VALDERREBOLLO/C/ GRANJA DE SAN ILDEFONSO/,59.0,1.000000,None,0.000022,None,"(40.3708906, -3.627407)"
9096,8,NOT,GRUTA DEL LABERINTO,PARQUE EL CAPRICHO,96.0,1.000000,"(Parque de El Capricho, Barrio del Aeropuerto,...",0.573331,"(40.4563176, -3.59797462561085)","(40.4561933, -3.5959202)"
8016,1,NaN,CARLOS III,JARDINES SABATINI,10.0,0.969697,"(Jardines de Sabatini, Cuesta de San Vicente, ...",0.044958,"(40.4204954, -3.71393477143546)","(40.4111512, -3.6907121)"
8974,1,NOT,CARLOS III,PUERTA DEL SOL,10.0,0.969697,"(Puerta del Sol, Calle del Carmen, Barrio Sol,...",0.079367,"(40.4170316, -3.7034288)","(40.4111512, -3.6907121)"
9649,6,NaN,ARBOL MEDIÁTICO,BULEVAR DE LA NATURALEZA Nº40 (SUR),60.0,0.955556,None,0.000022,None,"(40.3700085, -3.6301475)"
8261,1,NOT,CARLOS II,"PARQUE DEL RETIRO, (Pº. ARGENTINA)",10.0,0.939394,None,0.000022,None,"(40.4111512, -3.6907121)"
9189,1,NaN,GOYA,ALCALA C/V GOYA,20.0,0.933333,None,0.000022,None,"(40.4248513, -3.6753031)"


In [209]:
data['features'][59]['geometry']['coordinates'][1]

40.3708906

In [212]:
test.sort_values('OSMLocation_score', ascending=False).head()

,TIPO,CALIF,TITULO,LOCALIZACION,OSM,OSM_score,OSMLocation,OSMLocation_score,COORD,COORD_OSM
COD,,,,,,,,,,
8552,1,NaN,JAIME I,FRANCISCO Y JACINTO ALCANTARA,56.0,1.000000,"(Calle de Francisco y Jacinto Alcántara, Parqu...",2.681920,"(40.4295405, -3.7238498)","(40.429222, -3.7239872)"
8014,1,NOT,CALDERON DE LA BARCA,PZA. SANTA ANA,19.0,0.853050,"(Plaza Santa Ana, Calle del Prado, Barrio Cort...",2.671066,"(40.4147254, -3.7008044)","(40.4146613, -3.7012384)"
9179,1,NOT,FEDERICO GARCIA LORCA,PLAZA DE SANTA ANA,16.0,0.837157,"(Plaza de Santa Ana, Barrio Cortes, Centro, Ma...",1.390338,"(40.4148283, -3.7013386)","(40.4147407, -3.7004971)"
9074,8,NOT,ESTANQUES GEMELOS DEL PARTERRE,PARQUE EL CAPRICHO,94.0,0.850505,"(Parque de El Capricho, Barrio del Aeropuerto,...",0.619641,"(40.4563176, -3.59797462561085)","(40.4563558, -3.5960684)"
9096,8,NOT,GRUTA DEL LABERINTO,PARQUE EL CAPRICHO,96.0,1.000000,"(Parque de El Capricho, Barrio del Aeropuerto,...",0.573331,"(40.4563176, -3.59797462561085)","(40.4561933, -3.5959202)"
